In [1]:
! pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 66.0 MB/s eta 0:00:00


In [2]:
## pdf-Datei anschauen: Die ersten 12 Seiten enthalten Deckblatt + Inhaltsverzeichnis + Kurzfassung - das kann also übersprungen werden
## außerdem letzten beiden Seiten nur Wahlaufruf und Formalien

# Als erstes die Formatinformation extrahieren: Also Text mit Schriftgröße und Dicke der Schrift, um später die Überschriften zu finden und
# zur Gliederung verwenden zu können

# Text extrahieren inkl. Formatinfos:

import pymupdf as fitz

def extract_text_with_formatting(pdf_path, skip_pages=11):
    doc = fitz.open(pdf_path)
    text_data = []

    for page_num in range(skip_pages, 80):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]  # Extrahiert Textblöcke

        for block in blocks:
            if block["type"] == 0:  # Textblock
                for line in block["lines"]:
                    for span in line["spans"]:
                        # Speichere Text + Schriftgröße + Fettdruck-Info
                        text_data.append({
                            "text": span["text"],
                            "size": span["size"],  # Schriftgröße
                            "bold": span["flags"] & 2 != 0  # Prüft, ob der Text fett ist
                        })

    return text_data

# PDF-Dateipfad
pdf_path = "/content/Wahlprogramm_CDU.pdf"

# Text mit Formatierungsinformationen extrahieren
text_data = extract_text_with_formatting(pdf_path)

# Zeige die ersten paar Textabschnitte mit Formatierung
for item in text_data[:100]:
    print(f"Text: {item['text']}, Size: {item['size']}, Bold: {item['bold']}")


Text:  , Size: 11.039999961853027, Bold: False
Text: - 10 - , Size: 12.0, Bold: False
Text: - LANGFASSUNG - , Size: 11.039999961853027, Bold: False
Text: I., Size: 21.959999084472656, Bold: False
Text:  , Size: 21.959999084472656, Bold: False
Text: F, Size: 21.959999084472656, Bold: False
Text: F, Size: 21.959999084472656, Bold: False
Text: ür ein Deutschland, auf das wir , Size: 21.959999084472656, Bold: False
Text: wieder stolz sein können , Size: 21.959999084472656, Bold: False
Text: Unsere Bundesrepublik Deutschland ist eine großartige Erfolgsgeschichte: Wiederaufbau, , Size: 11.039999961853027, Bold: False
Text: Wirtschaftswunder und Weltmeistertitel, Westbindung, Friedliche Revolution, Wiederver-, Size: 11.039999961853027, Bold: False
Text: einigung und Aufbauleistung der Ostdeutschen, Wohlstand, Soziale Marktwirtschaft mit , Size: 11.039999961853027, Bold: False
Text: guter Arbeit und starken Sozialversicherungen, „Made in Germany“ als weltweit bekanntes , Size: 11.0399999618530

In [3]:
## Output anschauen: Normale Schrift ist in Schriftgröße 11,03999, Überschriften sind größer, Seitenzahlen stehen in einzelnen Zeilen im Format "- 10 -"
## einzelne Buchstaben in Zeilen etc.

import pandas as pd
import re

def split_sections(text_data, min_size=11.5):
    sections = []
    current_section = []

    def is_page_number(text, size):
        """Prüft, ob der Text eine Seitenzahl im Format '- 12 -' ist."""
        return size == 12 and re.match(r"^-\s*\d+\s*-$", text) is not None

    for item in text_data:
        text = item["text"].strip()

        if is_page_number(text, item["size"]): # Seitenzahlen überspringen
            continue

        if item["size"] > 12: #Überschriften überspringen
            continue

        if min_size < item["size"] < 12.5: #Überschriften + Abschnitt definieren
            if current_section:
                sections.append("".join(current_section).strip())
            current_section = []

        elif text == "":  # Leere Zeilen überspringen
            continue

        elif text.isupper():  # Großgeschriebene Wörter überspringen
            continue
        else:
            current_section.append(text)

    # Letzten Abschnitt hinzufügen
    if current_section:
        sections.append("".join(current_section).strip())

    return sections


# Abschnitte extrahieren
sections = split_sections(text_data)

# Tabelle
df = pd.DataFrame(sections, columns=["Abschnitt"])

print(df[:10])


                                           Abschnitt
0  Unsere Bundesrepublik Deutschland ist eine gro...
1  Arbeit ist sinnstiftend und schafft Wohlstand,...
2  Unser Arbeitsrecht ist in die Jahre gekommen, ...
3  Der Arbeits- und Fachkräftemangel bremst unser...
4  Deutschland ist für Unternehmen ein Hochsteuer...
5  Überbordende Dokumentations- und Meldepflichte...
6  Auch die Steuerbürokratie nimmt zu und ist ein...
7  Geoökonomische Herausforderungen und Krisen ne...
8  Exportweltmeister sind jetzt andere, nicht meh...
9  Die Energiekrise hat unser Land verändert. Höh...


In [4]:
# Speichern der Tabelle als CSV-Datei
df.to_csv("/content/cdu.csv", index=False, encoding="utf-8")